# AHLT - Second delivery - Task 9.2 DDI
**Albert Rial**   
**Karen Lliguin**   

This delivery consists of solving the task 9.3 of the SemEval-2013 challenge. The task concerns classifying drug-drug interactions between pairs of drugs. 

The dataset provided contains XML files with sentences and the drugs appearing on it and their corresponding interaction. There are four general types: mechanism, int, effect and advise. The data is already splitted in three subsets: Train, Devel and Test.



## Goal 1: DDI baseline
### Introduction

### Results

#### Devel

#### Test


### Conclusions

The final results obtained are:



## Goal 2: DDI using machine learning
### Introduction

### Results

#### Devel

#### Test


### Conclusions

The final results obtained are: